# Data Extraction

## Parsing Y- axis Ticks and Y- vals 
This code parses the x and y axes (horizontal and vertical axes) detection for the y-ticks, calculates the ratio to normalize the figure contents, specifically the bar plots with respect to the y-ticks and finally connects the evaluated y-values with their correspinding legends using clustering algorithm. 
* Note that all the figures recorded have their respective dois as a unique key of identification.

In [1]:
import cv2, imutils, re
import xlsxwriter, json, os
import matplotlib.pyplot as plt
import numpy as np
import pytesseract
from pathlib import Path
from matplotlib import rcParams
from pytesseract import Output

In [2]:
# Directory of images to run the code on
img_dir = 'AllData/processed_images/Part2'
data_dir = 'data'

# Directory to save the output images
save_dir = 'out'

### Using json to parse the captions and dois for each image

In [3]:
with open('urldata.json') as handler:
    urldata = json.load(handler)

In [4]:
with open('doi_data.json') as doihandler:
    doidata = json.load(doihandler)

In [5]:
with open('ocr-image-text.json') as ocrhandler:
    images_text = json.load(ocrhandler)

### Function to detect x-axis and y-axis for a plot

In [6]:
def findMaxConsecutiveOnes(nums) -> int:
    count = maxCount = 0
    
    for i in range(len(nums)):
        if nums[i] == 1:
            count += 1
        else:
            maxCount = max(count, maxCount)
            count = 0
                
    return max(count, maxCount)

In [7]:
def detectAxes(filepath, threshold=None, debug=False):
    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
    
    image = cv2.imread(filepath)
    height, width, channels = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[:, idx] < 200) for idx in range(width)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < width:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            break
            
        start_idx += 1
           
    yaxis = (maxindex, 0, maxindex, height)
    
    if debug:
        fig, ax = plt.subplots(1, 2)

        ax[0].imshow(image)

        ax[1].plot(maxConsecutiveOnes, color = 'k')
        ax[1].axhline(y = max(maxConsecutiveOnes) - 10, color = 'r', linestyle = 'dashed')
        ax[1].axhline(y = max(maxConsecutiveOnes) + 10, color = 'r', linestyle = 'dashed')
        ax[1].vlines(x = maxindex, ymin = 0.0, ymax = maxConsecutiveOnes[maxindex], color = 'b', linewidth = 4)

        plt.show()

    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[idx, :] < 200) for idx in range(height)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < height:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            
        start_idx += 1
            
    cv2.line(image, (0, maxindex), (width, maxindex),  (255, 0, 0), 2)
    xaxis = (0, maxindex, width, maxindex)
    
    if debug:
        rcParams['figure.figsize'] = 15, 8

        fig, ax = plt.subplots(1, 1)
        ax.imshow(image, aspect = 'auto')
        
    return xaxis, yaxis

### Here we try to get text from the image using pytesseract

In [8]:
def getTextFromImageArray(image, mode):
    image_text = []
    
    if mode == 'y-text':
        image = cv2.transpose(image)
        image = cv2.flip(image, flipCode = 1)
        config = "-l eng --oem 1 --psm 11"
    elif mode == 'y-labels':
        config = "-l eng --oem 1 --psm 6 -c tessedit_char_whitelist=.0123456789"
    
    d = pytesseract.image_to_data(image, config = config, output_type = Output.DICT)
    
    n_boxes = len(d['text'])

    # Pick only the positive confidence boxes
    for i in range(n_boxes):
            
        if int(d['conf'][i]) >= 0:
                
            text = d['text'][i].strip()
            
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            image_text.append((d['text'][i], (x, y, w, h)))
            
    # Remove all the duplicates in (text, box) pairs
    return list(set(image_text))

In [9]:
def getTextFromImage(filepath, bw=False, debug=False):
    image_text = []
    
    image = cv2.imread(filepath)
    height, width, _ = image.shape
        
    if bw:
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # define range of black color in HSV
        lower_val = np.array([0, 0, 0])
        upper_val = np.array([179, 255, 179])

        # Threshold the HSV image to get only black colors
        mask = cv2.inRange(hsv, lower_val, upper_val)

        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(image, image, mask = mask)

        # invert the mask to get black letters on white background
        image = cv2.bitwise_not(mask)
            
    d = pytesseract.image_to_data(image, config = "-l eng --oem 1 --psm 11", output_type = Output.DICT)
    n_boxes = len(d['text'])

    # Pick only the positive confidence boxes
    for i in range(n_boxes):
            
        if int(d['conf'][i]) >= 0:
                
            text = d['text'][i].strip()
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            image_text.append((d['text'][i], (x, y, w, h)))
     
    if bw:  
        image = cv2.imread(filepath)
        image_text = list(set(image_text))
        white_bg = 255 * np.ones_like(image)
        
        for text, (textx, texty, w, h) in image_text:
            roi = image[texty:texty + h, textx:textx + w]
            white_bg[texty:texty + h, textx:textx + w] = roi
            
        image_text = []
        d = pytesseract.image_to_data(white_bg, config = "-l eng --oem 1 --psm 11", output_type = Output.DICT)
        n_boxes = len(d['text'])

        # Pick only the positive confidence boxes
        for i in range(n_boxes):

            if int(d['conf'][i]) >= 0:

                text = d['text'][i].strip()
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                image_text.append((d['text'][i], (x, y, w, h)))
        
    # Remove all the duplicates in (text, box) pairs
    return list(set(image_text))

In [10]:
def getProbableLabels(image, image_text, xaxis, yaxis):
    y_labels = []
    x_labels = []
    legends = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            if bool(re.findall(r'\b[\d\.\d]+\b', text)):
                y_labels.append((text, (textx, texty, w, h)))
            
        # To the right of y-axis and bottom of x-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == 1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            x_labels.append((text, (textx, texty, w, h)))
            
        # Top of x-axis and to the right of y-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            
            # Consider non-numeric only for legends
            if not bool(re.findall(r'\b[\d\.\d]+\b', text)):
                legends.append((text, (textx, texty, w, h)))
            
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(y_labels):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(y_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
        
    y_labels = []
    for text, (textx, texty, w, h) in maxList:
        y_labels.append(text)
        #cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 0), 2)
        
    # Get the x-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(y1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_labels):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Sort bounding rects by x coordinate
    def getYFromRect(item):
        return item[1]

    maxList.sort(key = getYFromRect)
    y_labels_list = maxList.copy()
    
    x_labels = []
    for text, (textx, texty, w, h) in maxList:
        x_labels.append(text)
        #cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 0), 2)
    
    # Get possible legend text
    # For this, we need to search both top to
    # bottom and also from left to right.
    maxIntersection = 0
    maxList = []
    for i in range(y1):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
            
    for i in range(x11, width):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
        
    legends = []
    #legendBoxes = []
    for text, (textx, texty, w, h) in maxList:
        legends.append(text)
    #    legendBoxes.append((textx, texty, w, h))
        #cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 255), 2)
    
    #legendBoxes = mergeRects(legendBoxes)
    
    #for (textx, texty, w, h) in legendBoxes:
    #    cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 255), 2)
    
    return image, x_labels, y_labels, y_labels_list, legends

In [11]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [12]:
def maskImageForwardPass(filepath, start_idx):
    image = cv2.imread(filepath)
    height, width, channels = image.shape
        
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    start_idx = 1
    try:
        while start_idx <= width:
            if sum(gray[:, start_idx] < 200) != 0:
                break
            else:
                start_idx += 1

        end_idx = start_idx
        while end_idx <= width:
            if sum(gray[:, end_idx] < 200) == 0:
                break
            else:
                end_idx += 1
    except IndexError:
        pass

    gray[:, 1:start_idx] = 255
    gray[:, end_idx:width] = 255
        
    return gray, start_idx, end_idx

In [13]:
def maskImageBackwardPass(filepath, end_idx):
    image = cv2.imread(filepath)
    height, width, channels = image.shape

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    while end_idx > 0:
        if sum(gray[:, end_idx] < 200) == 0:
            break
        else:
            end_idx -= 1
        
    gray[:, end_idx:width] = 255
        
    return gray

## Getting the Ratio for y-value matching 

Similar to the label detection logic, y-ticks are detected: 

Y-ticks 

*  Check only the numerical boxes which are to the left of y-axis and to the top of x-axis.
*  Run a line sweep from left end of the image to the y-axis position, and check when the sweeping line intersects with the maximum number of numerical boxes.
* The numerical boxes are then used as bounding boxes for calculating the y-ticks.

Ratio:
* Difference between the y-ticks is then calculated.  
* Only consider the mean difference between the y-ticks, rejecting the outliers from the calculated values.
* The final ratio is: <br> ``normalize_ratio := ticks_diff.mean() / y-ticks.mean()``.

In [14]:
def getRatio(path, xaxis, yaxis):
    list_text = []
    list_ticks = []
    
    filepath = img_dir + "/" + path.name
    
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels = image.shape

    image_text = images_text[path.name]
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
                    
        (x1, y1, x2, y2) = xaxis
        (x11, y11, x22, y22) = yaxis
        
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            # Consider numeric only for ticks on y-axis
            numbers = re.findall(r'\d+(?:\.\d+)?', text)
            if bool(numbers):
                list_text.append((numbers[0], (textx, texty, w, h)))
                          
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(list_text):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(list_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Get list of text and ticks
    list_text = []
    for text, (textx, texty, w, h) in maxList:
        list_text.append(float(text))
        list_ticks.append(float(texty + h))
        
    text_sorted = (sorted(list_text))
    ticks_sorted  = (sorted(list_ticks))
    
    ticks_diff = ([ticks_sorted[i] - ticks_sorted[i-1] for i in range(1, len(ticks_sorted))])
    text_diff = ([text_sorted[i] - text_sorted[i-1] for i in range(1, len(text_sorted))])
    print("[get text-to-tick ratio] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    # Detected text may not be perfect! Remove the outliers.
    ticks_diff = reject_outliers(np.array(ticks_diff), m=1)
    text_diff = reject_outliers(np.array(text_diff), m=1)
    print("[reject_outliers] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()

    return text_sorted, normalize_ratio

In [15]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [16]:
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) <= m * np.std(data)]

## Writing to Excel workbook

In [17]:
def addToExcel(dataname, data, row):
    col = 0

    worksheet.write(row, col, dataname)
    for content in data:
        col += 1
        worksheet.write(row, col, content)

## Matching the ratio for final data extraction 

Y-val data:
* The height of each bounding box is calculated by cluster count estimation method. 
* This ratio previously calculated is then used to calculate the <br> ``y_values :=  v_value = normalize_ratio x height of bounding box``.

In [18]:
def mergeRects(contours):
    rects = []
    rectsUsed = []

    # Just initialize bounding rects and set all bools to false
    for cnt in contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[0]

    rects.sort(key = getXFromRect)

    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 5
    yThr = 5

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[0]
            currxMax = supVal[0] + supVal[2]
            curryMin = supVal[1]
            curryMax = supVal[1] + supVal[3]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[0]
                candxMax = subVal[0] + subVal[2]
                candyMin = subVal[1]
                candyMax = subVal[1] + subVal[3]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)

                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    #for rect in acceptedRects:
    #    img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
    
    return acceptedRects

In [19]:
def nearbyRectangle(current, candidate, threshold):
    (currx, curry, currw, currh) = current
    (candx, candy, candw, candh) = candidate
    
    currxmin = currx
    currymin = curry
    currxmax = currx + currw
    currymax = curry + currh
    
    candxmin = candx
    candymin = candy
    candxmax = candx + candw
    candymax = candy + candh
    
    # If candidate is on top, and is close
    if candymax <= currymin and candymax + threshold >= currymin:
        return True
    
    # If candidate is on bottom and is close
    if candymin >= currymax and currymax + threshold >= candymin:
        return True
    
    # If intersecting at the top, merge it
    if candymax >= currymin and candymin <= currymin:
        return True
    
    # If intersecting at the bottom, merge it
    if currymax >= candymin and currymin <= candymin:
        return True
    
    # If intersecting on the sides or is inside, merge it
    if (candymin >= currymin and
        candymin <= currymax and
        candymax >= currymin and
        candymax <= currymax):
        return True
    
    return False

In [20]:
def euclidean(v1, v2):
    return sum((p - q) ** 2 for p, q in zip(v1, v2)) ** .5

def angle_between(p1, p2):
    
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]

    #print(deltaX)
    #print(deltaY)
    return math.atan2(deltaY, deltaX) / math.pi * 180
    
def RectDist(rectA, rectB):
    (rectAx, rectAy, rectAw, rectAh) = rectA
    (rectBx, rectBy, rectBw, rectBh) = rectB
    
    angle = angle_between((rectAx + rectAw / 2, rectAy + rectAh / 2), (rectBx + rectBw / 2, rectBy + rectBh / 2))
    
    if (angle >= -5 and angle <= 5) or (angle <= -175 and angle >= -185) or (angle >= 175 and angle <= 185):
        if (rectBx > rectAx + rectAw):
            #print(rectA)
            #print(rectB)
            #print(angle)
            return euclidean([rectAx + rectAw / 2, rectAy + rectAh / 2], [rectBx + rectBw / 2, rectBy + rectBh / 2])
        else:
            return sys.maxsize
    else:
        return sys.maxsize

In [21]:
def getProbableYLabels(image, contours, xaxis, yaxis):
    y_labels = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, rect in enumerate(contours):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(contours[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
                    
    return image, maxList

## Saving y-values in our data excel sheet 

In [22]:
import ast, cv2, re, sys
import math
import json


images = []
texts = []

def getYVal(img_dir):
    yValueDict = {}
    for index, path in enumerate(Path(img_dir).iterdir()):
        if path.name.endswith('.png') or path.name.endswith('.jpg'):
            filepath = img_dir + "/" + path.name
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            xaxis, yaxis = detectAxes(filepath)
            
            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)

            #image_text = getTextFromImage(filepath, bw=True)
            image_text = images_text[path.name]
            img, x_labels, _, _, legends = getProbableLabels(img, image_text, xaxis, yaxis)
            try:
                list_text, normalize_ratio = getRatio(path, xaxis, yaxis)
                print("[{0}] path: {1}, ratio: {2}".format(index, path.name, normalize_ratio), end='\n\n')

                #rcParams['figure.figsize'] = 15, 8
                #plt.imshow(img)

                d = pytesseract.image_to_data(img, config = "-l eng --oem 1 --psm 11", output_type = Output.DICT)
                n_boxes = len(d['text'])

                # Color detection
                h, w, _ = np.shape(img)

                image = img.reshape((h * w, 3))

                # Remove white and near-by pixels
                threshold = 5
                for r in range(255 - threshold, 256):
                    for g in range(255 - threshold, 256):
                        for b in range(255 - threshold, 256):
                            image = image[np.where((image != [r, g, b]).any(axis = 1))]

                values, counts = np.unique(image, axis = 0, return_counts = True)

                greater_counts = np.where(counts >= 100)
                values, counts = values[greater_counts], counts[greater_counts]

                sort_indices = np.argsort(-counts)
                values, counts = values[sort_indices], counts[sort_indices]

                groups = []
                groupcounts = []

                for idx, value in enumerate(values):
                    grouped = False

                    for groupid, group in enumerate(groups):
                        for member in group:
                            r, g, b = member
                            vr, vg, vb = value

                            if (abs(vr.astype(np.int16) - r.astype(np.int16)) <= 5 and
                                abs(vg.astype(np.int16) - g.astype(np.int16)) <= 5 and
                                abs(vb.astype(np.int16) - b.astype(np.int16)) <= 5):
                                group.append(value)
                                groupcounts[groupid] += counts[idx]
                                grouped = True
                                break

                        if grouped:
                            break

                    if not grouped:
                        groups.append([value])
                        groupcounts.append(counts[idx])

                groupcounts = np.array(groupcounts)
                sort_indices = np.argsort(-groupcounts)
                new_groups = [groups[i] for i in sort_indices]
                groups = new_groups
                
                for i in range(len(groups[:4])):

                    img = cv2.imread(img_dir + "/" + path.name)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    mask = None
                    for value in groups[i]:
                        COLOR_MIN = np.array([value[0], value[1], value[2]], np.uint8)
                        COLOR_MAX = np.array([value[0], value[1], value[2]], np.uint8)

                        if mask is None:
                            mask = cv2.inRange(img, COLOR_MIN, COLOR_MAX)
                        else:
                            mask = mask | cv2.inRange(img, COLOR_MIN, COLOR_MAX)

                    image = cv2.bitwise_and(img, img, mask = mask)
                    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (3, 3))

                    edged = cv2.Canny(image, 0, 250)
                    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours = [contour for contour in contours if cv2.contourArea(contour) > 0.]

                    # Remove noisy ones!
                    if len(contours) == 0 or len(contours) > 100:
                        continue

                    contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]

                    rects = mergeRects(contours)
                    textBoxes = []
                    legends = []
                    
                    for rectBox in rects:
                        min_distance = sys.maxsize
                        closestBox = None
                        legendtext = None

                        for text, textBox in image_text:    
                            if RectDist(rectBox, textBox) < min_distance:
                                closestBox = textBox
                                min_distance = RectDist(rectBox, textBox)
                                legendtext = text

                        if closestBox is not None:
                            textBoxes.append(closestBox)

                        if legendtext is not None:
                            legends.append(legendtext)

                    list_len = []
                    
                    for rect in rects:
                        #img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
                        list_len.append(float(rect[3]))

                    # y-values will be a product of the normalize ratio and each length              
                    y_val = [round(l* normalize_ratio, 1) for l in list_len]

                    if path.name in yValueDict:
                        if legends:
                            yValueDict[path.name].append([legends[-1]])
                        else:
                            yValueDict[path.name].append([''])
                            
                        yValueDict[path.name].append(y_val)
                    else:
                        if legends:
                            yValueDict[path.name] = [legends[-1]]
                        else:
                            yValueDict[path.name] = ['']
                            
                        yValueDict[path.name].append(y_val)
                        
                    for box in textBoxes:
                        (x, y, w, h) = box
                        #cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    #for (x1, y1, x2, y2) in [xaxis]:
                    #    cv2.line(img, (x1, y1), (x2, y2),  (0, 0, 255), 2)

                    #for (x1, y1, x2, y2) in [yaxis]:
                    #    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        
            except Exception as e:
                print(e)
                continue
            
    return yValueDict

In [23]:
yValueDict = getYVal(img_dir)

[get text-to-tick ratio] ticks_diff: [44.999994307756424, 44.000006437301636, 46.99999940395355, 44.000006437301636, 45.99999392032623, 44.000006437301636], text_diff: [15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
[reject_outliers] ticks_diff: [44.99999431 44.00000644 44.00000644 45.99999392 44.00000644], text_diff: [15. 15. 15. 15. 15. 15.]
[0] path: 08813216-Figure1-1.png, ratio: 0.33632285858354183

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1] path: 08813216-Figure4-1.png, ratio: nan



C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\dsand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:62: RuntimeWar

[get text-to-tick ratio] ticks_diff: [34.000000562518835, 33.999999821186066, 33.99999389052391, 33.999999821186066, 35.00000435113907, 33.00000715255737, 32.99998342990875, 35.00001621246338, 33.999987959861755], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [34.00000056 33.99999982 33.99999389 33.99999982 33.99998796], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[2] path: 1-s2.0-S0009250913008336-main-Figure10-1.png, ratio: 0.00294117678105061

[get text-to-tick ratio] ticks_diff: [34.00000225007534, 33.99999922513962, 31.999996423721313, 33.99999922513962, 32.0000085234642, 34.999994575977325, 19.000008463859558, 46.99999928474426, 31.999996423721313, 32.99999177455902], text_diff: [0.1, 0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009,

[get text-to-tick ratio] ticks_diff: [33.00000078231096, 33.99999716877937, 33.99999716877937, 33.0000074505806, 34.00000250339508, 32.99998611211777, 34.00000250339508, 32.99999678134918, 34.00000250339508, 32.99999678134918], text_diff: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[reject_outliers] ticks_diff: [33.00000078 33.99999717 33.99999717 33.00000745], text_diff: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[22] path: 1-s2.0-S0016236115011084-main-Figure5-1.png, ratio: 0.029850745696045834

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[23] path: 1-s2.0-S0016236115011084-main-Figure6-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [71.00000170059502, 71.00000254809856, 69.99999582767487, 72.00000926852226, 69.99999582767487, 70.99999576807022], text_diff: [0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009]
[reject_outliers] ticks_diff: [71.00000

[get text-to-tick ratio] ticks_diff: [76.00000440329313, 75.00000184774399, 75.99999660253525, 75.00000184774399, 76.99999135732651, 73.9999862909317], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [76.0000044  75.00000185 75.9999966  75.00000185], text_diff: [10. 10. 10. 10. 10. 10.]
[43] path: 1-s2.0-S0016236117309912-main-Figure1-1.png, ratio: 0.13245032906393742

[get text-to-tick ratio] ticks_diff: [67.99999647773802, 66.99999570846558, 66.00000813603401], text_diff: [2.0, 2.0, 2.0]
[reject_outliers] ticks_diff: [66.99999571], text_diff: [2. 2. 2.]
[44] path: 1-s2.0-S0016236117315831-main-Figure1-1.png, ratio: 0.029850748180679305

[get text-to-tick ratio] ticks_diff: [55.99999946728349, 58.99999983608723, 55.99999339878559, 44.9999994635582, 70.00000995397568, 56.99999284744263, 58.00001657009125], text_diff: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[reject_outliers] ticks_diff: [55.99999947 58.99999984 55.9999934  56.99999285 58.00001657], text_diff: [

...

[get text-to-tick ratio] ticks_diff: [31.99999973922968, 32.00000183284283, 32.99999521672726, 32.00000602006912, 31.99999764561653, 32.00000602006912, 31.99998927116394], text_diff: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
[reject_outliers] ticks_diff: [31.99999974 32.00000183 32.00000602 31.99999765 32.00000602 31.99998927], text_diff: [100. 100. 100. 100. 100. 100. 100.]
[1145] path: ep.11814-Figure6-1.png, ratio: 3.124999991390117

[get text-to-tick ratio] ticks_diff: [37.999998301267624, 38.00000350177288, 38.00000350177288, 37.999998301267624, 37.99998790025711, 37.00001394748688, 37.99998790025711, 38.00000870227814], text_diff: [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
[reject_outliers] ticks_diff: [37.9999983 38.0000035 38.0000035 37.9999983 37.9999879 37.9999879
 38.0000087], text_diff: [50. 50. 50. 50. 50. 50. 50. 50.]
[1146] path: ep.11847-Figure1-1.png, ratio: 1.3157895325045863

[get text-to-tick ratio] ticks_diff: [38.00000290572643, 35.000000193715096, 3

[get text-to-tick ratio] ticks_diff: [30.999998960644007, 31.999999970197678, 30.000002801418304, 31.00000461935997, 30.999998152256012], text_diff: [20.0, 20.0, 20.0, 20.0, 20.0]
[reject_outliers] ticks_diff: [30.99999896 31.00000462 30.99999815], text_diff: [20. 20. 20. 20. 20.]
[1172] path: High quality fuel from food waste investigation of a stepwise process from the perspective of technology development-Figure2-1.png, ratio: 0.6451612783055154

[get text-to-tick ratio] ticks_diff: [46.99999830685556, 46.00000612437725, 46.00000038743019, 92.99998506903648, 46.00001186132431], text_diff: [10.0, 20.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [46.99999831 46.00000612 46.00000039 46.00001186], text_diff: [10. 10. 10. 10.]
[1173] path: Hydrothermal catalytic liquefaction mechanisms of agal biomass to bio oil-Figure2-1.png, ratio: 0.21621619672171125

[get text-to-tick ratio] ticks_diff: [21.999998182058334, 22.000004693865776, 24.000002160668373, 21.999998182058334, 20.999992936

[get text-to-tick ratio] ticks_diff: [37.999999672174454, 40.00000223517418, 37.999999672174454, 39.000000953674316, 37.99999266862869, 39.00001496076584, 37.99997866153717, 39.000000953674316, 38.000006675720215], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [37.99999967 37.99999967 39.00000095 37.99999267 39.00001496 37.99997866
 39.00000095 38.00000668], text_diff: [5. 5. 5. 5. 5. 5. 5. 5. 5.]
[1201] path: Minarick2011_Article_ProductAndEconomicAnalysisOfDi-Figure1-1.png, ratio: 0.13029316206289748

[get text-to-tick ratio] ticks_diff: [67.00000373832881, 0.9999980628490448, 55.999995931982994, 80.00000566244125, 66.99998265504837, 67.0000147819519], text_diff: [0.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [67.00000374 55.99999593 80.00000566 66.99998266 67.00001478], text_diff: [10. 10. 10. 10. 10.]
[1202] path: Minarick2011_Article_ProductAndEconomicAnalysisOfDi-Figure2-1.png, ratio: 0.1483679513028409

[get text-to-tic

[get text-to-tick ratio] ticks_diff: [49.999999046325684, 50.000004529953, 47.99999952316284, 49.999988079071045, 49.999999046325684, 49.00000476837158], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [49.99999905 50.00000453 49.99998808 49.99999905 49.00000477], text_diff: [5. 5. 5. 5. 5. 5.]
[1224] path: Saber2018_Article_UltrasonicPretreatmentForLow-t-Figure3-1.png, ratio: 0.10040160825226734

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1225] path: Saoud2018_Chapter_NanocatalystForBiofuelProducti-Figure3-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1226] path: sc400145w-Figure2-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1227] path: sc400145w-Figure4-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] tick

[get text-to-tick ratio] ticks_diff: [35.00000083446503, 35.99999988079071, 35.99999499320984, 35.99999499320984, 35.0000057220459, 36.00000476837158], text_diff: [20.0, 20.0, 20.0, 20.0, 20.0, 20.0]
[reject_outliers] ticks_diff: [35.99999988 35.99999499 35.99999499 36.00000477], text_diff: [20. 20. 20. 20. 20. 20.]
[1250] path: wene.319-Figure5-1.png, ratio: 0.5555555762516136

[get text-to-tick ratio] ticks_diff: [36.00000038743019, 36.99999779462814, 35.99999785423279, 36.0000079870224, 110.99998831748962], text_diff: [10.0, 0.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [36.00000039 36.99999779 35.99999785 36.00000799], text_diff: [10. 10. 10. 10.]
[1251] path: wene.319-Figure6-1.png, ratio: 0.2758620613111754

[get text-to-tick ratio] ticks_diff: [53.00000309944153, 159.99999567866325], text_diff: [60.0, 20.0]
[reject_outliers] ticks_diff: [ 53.0000031  159.99999568], text_diff: [60. 20.]
[1252] path: Xie2019_Article_DewaterabilityEnhancementAndHe-Figure5-1.png, ratio: 0.375

In [24]:
workbook = xlsxwriter.Workbook('FigureData1.xlsx')

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg'):
        if path.name in yValueDict:
            filepath = img_dir + "/" + path.name
            print("[{0}] path: {1}".format(index, path.name))
            
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            height, width, channels = image.shape
            xaxis, yaxis = detectAxes(filepath)
            y_text = []

            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)
                
            #rcParams['figure.figsize'] = 15, 4
            #fig, ax = plt.subplots(1, 3)

            image_text = images_text[path.name]
            image, x_labels, y_labels, y_labels_list, legends = getProbableLabels(image, image_text, xaxis, yaxis)
            
            # Make a backward pass to skip the ticks
            gray = maskImageBackwardPass(filepath, yaxis[0])

            # Processing the image so that the contours are drawn to labels and text
            ret, thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
            rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 15))
            thresh = cv2.dilate(thresh, rect_kernel, iterations = 1)
            rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 1))
            thresh = cv2.dilate(thresh, rect_kernel, iterations = 1)

            contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = contours[0] if len(contours) == 2 else contours[1]
            rects = [cv2.boundingRect(contour) for contour in contours]
            print("number of contours: ", len(contours))

            #image_text = getTextFromImage(filepath, bw=True)
            image_text = images_text[path.name]
            #white_bg = 255 * np.ones_like(gray.copy())

            #for text, (textx, texty, w, h) in y_labels_list:
            #    roi = gray[texty:texty + h, textx:textx + w]
            #    white_bg[texty:texty + h, textx:textx + w] = roi

            # Get y-labels
            #y_labels_list = getTextFromImageArray(white_bg, 'y-labels')
            #ax[0].imshow(white_bg, aspect = 'auto')

            # Sort bounding rects by y coordinate
            def getYFromRect(item):
                return item[1][1]

            # Whiten y-labels and get y-text
            y_labels_list.sort(key = getYFromRect)
            #y_labels = []
            for text, (textx, texty, w, h) in y_labels_list:
                textx, texty, w, h = int(round(textx)), int(round(texty)), int(round(w)), int(round(h))
                roi = 255 * np.ones_like(gray[texty:texty + h, textx:textx + w])
                gray[texty:texty + h, textx:textx + w] = roi
                #y_labels.append(text)

            y_text_list = getTextFromImageArray(gray, 'y-text')

            # Sort bounding rects by x coordinate
            def getXFromRect(item):
                return item[1][0]

            y_text_list.sort(key = getXFromRect)        
            for text, (textx, texty, w, h) in y_text_list:
                y_text.append(text)

            # Get the remaining: x-labels and legends
            #image_text = getTextFromImage(filepath, bw=True)
            
            # Append doi values for each image.
            if path.name[:-16]+'.pdf' in doidata['Dois']:
                doi = doidata['Dois'][path.name[:-16]+'.pdf']
            else:
                doi = ''
             
            # Write to Excel
            worksheet = workbook.add_worksheet()            
            
            #addToExcel("caption", [urldata[path.name]], 0)
            addToExcel("doi",doi, 0)
            addToExcel("file name", [path.name[::-4] + ".pdf"], 1)
            addToExcel("x-labels", x_labels, 2)
            addToExcel("y-text", y_text, 3)
            addToExcel("y-labels", y_labels, 4)
            addToExcel("legends", legends, 5)
            
            for col_num, data in enumerate(yValueDict[path.name]):
                print(col_num, data)
                if type(data) != list:
                    worksheet.write(9 + col_num, 0, "Y-values:" + "\n\n" + str(data))
                else:
                    for pos,d in enumerate(data):
                        worksheet.write(9 + col_num, pos, str(d))

            # Print the output here!
            print("file name    :  ", path.name)
            print("doi          :  ", doi)
            print("x-labels     :  ", x_labels)
            print("y-text       :  ", y_text)
            print("y-labels     :  ", y_labels)
            print("legends      :  ", legends)
            print("Y- Values    :  ", ' '.join([str(elem) for elem in yValueDict[path.name]]), end = "\n\n")

            #for (x1, y1, x2, y2) in [xaxis]:
            #    cv2.line(image, (x1, y1), (x2, y2),  (0, 0, 255), 2)

            #for (x1, y1, x2, y2) in [yaxis]:
            #    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            #for text, (textx, texty, w, h) in y_labels_list:
            #    cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 255), 2)

            #ax[1].imshow(gray, aspect = 'auto')
            #ax[2].imshow(image, aspect = 'auto')

            # Insert the image
            worksheet.insert_image('J21', filepath)

# Close the excel workbook!
workbook.close()

[0] path: 08813216-Figure1-1.png
number of contours:  7
0 100.2
1 [4.4, 35.3, 0.7, 35.0, 28.9, 30.9, 37.7]
2 ['0.1']
3 [4.7, 26.9, 25.2, 26.6, 22.2, 30.3, 27.2]
4 ['A']
5 [17.2, 20.9, 25.9, 4.7, 1.7, 20.2, 19.5]
6 ['Solid']
7 [21.2, 18.8, 17.8, 18.5, 15.5, 4.7, 15.5]
file name    :   08813216-Figure1-1.png
doi          :   
x-labels     :   ['270 0', "290'C", '310 C', "330'C", '350']
y-text       :   ['7', 'Percentage', 'o', '*', 'of', 'prod', 'o', 'lucts', '(%)', '©']
y-labels     :   ['105', '90', '75', '60', '45', '30', '15']
legends      :   ['Aqueous', 'Phase', 'Gaseous', 'Phase', 'Solid', 'Residue']
Y- Values    :   100.2 [4.4, 35.3, 0.7, 35.0, 28.9, 30.9, 37.7] ['0.1'] [4.7, 26.9, 25.2, 26.6, 22.2, 30.3, 27.2] ['A'] [17.2, 20.9, 25.9, 4.7, 1.7, 20.2, 19.5] ['Solid'] [21.2, 18.8, 17.8, 18.5, 15.5, 4.7, 15.5]

[1] path: 08813216-Figure4-1.png
number of contours:  9
0 
1 [nan]
2 ['']
3 [nan, nan]
4 ['B']
5 [nan]
file name    :   08813216-Figure4-1.png
doi          :   
x-labels    

number of contours:  7
0 H2
1 [89.2, 2.3, 6.0, 10.9, 2.0, 0.3, 95.5, 95.3, 0.3, 8.9, 0.6, 0.9, 100.4, 2.3, 99.0, 2.0, 101.6, 3.7]
2 ['C02']
3 [0.3, 1.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 100.7, 0.3, 1.4, 0.3, 0.3, 0.3, 100.7, 0.3]
4 ['']
5 [92.1, 1.4, 1.2, 95.5]
6 ['']
7 [0.3, 0.3, 0.3, 2.0, 0.3, 71.1, 9.5, 1.7, 0.3, 92.1, 1.7, 2.0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
file name    :   1-s2.0-S0016236112010988-main-Figure4-1.png
doi          :   
x-labels     :   ['0.013', '0.025', '0.05', '0.075', '0.1']
y-text       :   ['2', '8', '&', 'Percentage', '3', '8', '3']
y-labels     :   ['100', '80', '20', '0']
legends      :   ['C2-C5', 'CO', 'B', 'H2']
Y- Values    :   H2 [89.2, 2.3, 6.0, 10.9, 2.0, 0.3, 95.5, 95.3, 0.3, 8.9, 0.6, 0.9, 100.4, 2.3, 99.0, 2.0, 101.6, 3.7] ['C02'] [0.3, 1.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 100.7, 0.3, 1.4, 0.3, 0.3, 0.3, 100.7, 0.3] [''] [92.1, 1.4, 1.2, 95.5] [''] [0.3, 0.3, 0.3, 2.0, 0.3, 71.1, 9.5, 1.7, 0.3,

number of contours:  7
0 paraffin
1 [1.9, 3.1, 2.8, 1.3, 2.2, 1.3, 1.9, 1.3]
2 ['(C)']
3 [1.3, 0.6, 0.6, 0.6, 0.6, 0.9, 0.6, 0.6, 1.6, 0.6, 0.9, 1.9, 1.6, 1.3, 2.8, 2.2, 1.6, 2.5]
4 ['300']
5 [0.3, 0.3, 1.6, 0.3, 0.3, 0.6, 0.9, 0.3, 0.3, 0.3, 0.6, 0.3, 0.3, 0.6, 0.3, 0.9, 0.3, 0.3, 0.3, 0.3, 1.9, 0.3, 0.3, 0.3, 0.3, 0.9, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.6, 3.1, 0.3, 0.6, 0.6, 0.3, 0.6, 2.8, 1.6, 0.3, 0.3, 3.1, 1.3, 0.3, 1.6, 0.6, 0.3, 0.3, 1.6, 2.8, 2.5]
file name    :   1-s2.0-S0016236115011345-main-Figure2-1.png
doi          :   
x-labels     :   ['250', '270', '290', '300']
y-text       :   ['°', 'Conversion', '8', 'x', 'of', 'PA', '&', '3', 'and', 'yield', '2', '8', 'of', 'parattins', '8', '8', '(C', "'/)", '8', '3']
y-labels     :   ['100', '80', '20']
legends      :   ['Yield', 'of', 'paraffin']
Y- Values    :   paraffin [1.9, 3.1, 2.8, 1.3, 2.2, 1.3, 1.9, 1.3] ['(C)'] [1.3, 0.6, 0.6, 0.6, 0.6, 0.9, 0.6, 0.6, 1.6, 0.6, 0.9, 1.9, 1.6, 1.3, 2.8, 2.2, 1.6, 2.5] ['300'] [0.3, 0.3

number of contours:  1
0 Sample F
1 [0.7, 2.3, 2.9, 5.9, 103.5, 3.6, 1.6, 3.6, 101.6, 2.9, 104.8, 1.0, 2.3, 3.3, 2.6, 2.3, 104.8, 2.3, 3.9, 4.2, 101.9, 3.3, 1.6, 2.6, 101.6, 2.0, 47.3, 2.6, 2.6, 11.1, 16.7, 2.0, 0.3, 1.6, 3.3, 10.1, 103.2]
file name    :   1-s2.0-S0016236117301709-main-Figure5-1.png
doi          :   
x-labels     :   ['ample E']
y-text       :   ['Liquefaction', 'yields']
y-labels     :   ['120%', '110%', '100%', '90%', '80%', '60%', '40%', '30%', '20%', '10%', '0%']
legends      :   ['Bio-crude', 'oil', 'Solid', 'residue', 'Aqueous', 'phase', '&']
Y- Values    :   Sample F [0.7, 2.3, 2.9, 5.9, 103.5, 3.6, 1.6, 3.6, 101.6, 2.9, 104.8, 1.0, 2.3, 3.3, 2.6, 2.3, 104.8, 2.3, 3.9, 4.2, 101.9, 3.3, 1.6, 2.6, 101.6, 2.0, 47.3, 2.6, 2.6, 11.1, 16.7, 2.0, 0.3, 1.6, 3.3, 10.1, 103.2]

[35] path: 1-s2.0-S0016236117305367-main-Figure4-1.png
number of contours:  7
0 450-500
1 [0.2, 0.3, 0.2, 0.2, 27.0, 0.7, 0.7, 4.6, 0.7, 0.7, 0.6, 0.4, 4.0, 11.7, 12.5, 0.2, 0.4, 21.0, 0.4, 4.6, 11

number of contours:  8
0 Gas,
1 [11.7, 11.8, 3.3, 8.9]
2 ['Bio-residue']
3 [0.2, 0.2, 0.2, 0.4, 0.2, 1.2, 0.2, 0.2, 0.2, 1.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 2.1, 0.2, 0.2, 0.6, 0.2, 0.2, 0.2, 0.2, 0.2, 67.2, 0.2, 1.7, 0.2, 1.0, 0.2, 0.2, 2.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 1.2, 0.2, 2.1, 0.2, 0.2, 1.2, 0.2, 0.2, 1.0, 0.2, 0.4, 0.4, 0.2, 0.2, 0.2, 1.2]
4 ['Temperature']
5 [0.4, 3.9, 1.0, 1.4, 1.7, 1.0, 1.7, 1.4]
file name    :   1-s2.0-S0016236118303363-main-Figure3-1.png
doi          :   
x-labels     :   ['260', '280', '300']
y-text       :   ['°', 'é', '=', '8', 'Products', '8', 'yield,', '&', '%', '&', '8', '8']
y-labels     :   ['60', '50', '40', '20', '10', '0']
legends      :   ['Bio-oil,', 'Gas,', 'Bio-residue']
Y- Values    :   Gas, [11.7, 11.8, 3.3, 8.9] ['Bio-residue'] [0.2, 0.2, 0.2, 0.4, 0.2, 1.2, 0.2, 0.2, 0.2, 1.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 2.1, 0.2, 0.2, 0.6, 0.2, 0.2, 0.2, 0.2, 0.2, 67.2, 0.2, 1.7, 0.2, 1.0, 0.2, 0.2, 2.3, 0.2, 

...

number of contours:  10
0 K2C03
1 [33.7, 35.0, 33.7, 32.8, 1.7]
2 ['K2C03']
3 [32.6, 30.9, 28.2, 34.9, 1.7]
4 ['NaOH']
5 [32.2, 31.2, 31.4, 25.7, 1.7]
file name    :   Midgett2012_Article_AssessingFeedstocksAndCatalyst-Figure4-1.png
doi          :   
x-labels     :   ['2', '3', '4']
y-text       :   ['ous', 'ligher', 'Heating', '=', '8', '8', 'Value', 'a', 'Bw', '(MJ/kg)', '&', '&']
y-labels     :   ['40', '35', '30', '25', '20', '15', '10']
legends      :   ['No', 'Na2C03', 'K2C03', 'NaOH']
Y- Values    :   K2C03 [33.7, 35.0, 33.7, 32.8, 1.7] ['K2C03'] [32.6, 30.9, 28.2, 34.9, 1.7] ['NaOH'] [32.2, 31.2, 31.4, 25.7, 1.7]

[1198] path: Midgett2012_Article_AssessingFeedstocksAndCatalyst-Figure5-1.png
number of contours:  2
0 K2C03
1 [157.9, 151.6, 113.7, 139.8, 8.1]
2 ['K2C03']
3 [131.7, 148.0, 70.4, 122.7, 8.1]
4 ['K2C03']
5 [113.7, 90.2, 87.5, 70.4, 8.1]
file name    :   Midgett2012_Article_AssessingFeedstocksAndCatalyst-Figure5-1.png
doi          :   
x-labels     :   ['1', '2', '3', 

number of contours:  8
0 dldl
1 [31.3, 56.1, 19.0, 20.2, 1.6, 21.5, 27.7, 26.3]
file name    :   Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.8-1.png
doi          :   
x-labels     :   ['Acorn', 'Tomato', 'Extracted', 'Hazelnut', 'Corncob', 'Sunflower']
y-text       :   ['°', 'FO', '2', '8', '8', 'IIIS', '&', '3', '2', 'g']
y-labels     :   ['60', '50', '40', '30', '20', '10']
legends      :   ['No', 'KCO3', 'Trona']
Y- Values    :   dldl [31.3, 56.1, 19.0, 20.2, 1.6, 21.5, 27.7, 26.3]

[1210] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.9-1.png
number of contours:  6
0 
1 [nan, nan, nan, nan, nan, nan, nan]
2 ['']
3 [nan, nan, nan]
file name    :   Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.9-1.png
doi          :   
x-labels     :   ['NaOH', 'KOH', 'Ca(OH)2', 'Na,CO3', 'K2CO3', 'NaHCO3', 'NaOH+', 'H2O,']
y-text       :   ['s', 'Hydrogen', 'yield,', 'a', 'mol/kg', '=', '5']
y-labels     :   ['16']
legends      :   []
Y- Values    

number of contours:  6
0 Grounds
1 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
file name    :   sc400145w-Figure2-1.png
doi          :   
x-labels     :   ['Bio-Oil', 'Biochar', 'Aqueous', 'Gaseous']
y-text       :   ['Percent', 'Mass', 'of', 'Dry', 'Wt.', '(%;']
y-labels     :   ['40']
legends      :   ['Spent', 'Grounds']
Y- Values    :   Grounds [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

[1227] path: sc400145w-Figure4-1.png
number of contours:  0
0 
1 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
file name    :   sc400145w-Figure4-1.png
doi          :   
x-labels     :   ['Defatted', 'Grounds']
y-text       :   []
y-labels     :   []
legends      :   []
Y- Values    :    [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

[1228] path: sc400145w-Figure5-1.png
number of contours:  0
0 
1 [nan, nan, nan, nan, nan, nan, nan, nan]
file name    :   sc400145w-Figure5-1.p

number of contours:  8
0 
1 [5.9, 13.6, 7.7, 3.0, 20.1, 10.6, 9.5, 3.3, 4.2, 4.9, 3.0, 1.7, 0.8]
2 ['300C']
3 [9.8, 13.6, 8.3, 2.0, 27.6, 9.6, 6.9, 1.7, 4.2, 3.7, 1.7, 0.3, 0.8]
4 ['']
5 [3.2, 8.3, 7.7, 3.6, 11.6, 10.9, 13.0, 5.9, 2.5, 5.2, 4.8, 4.4, 0.8]
file name    :   Sudasinghe2015_Article_Temperature-DependentLipidConv-Figure4-1.png
doi          :   
x-labels     :   ['N1', 'N1', 'N1', 'N2', 'N2', 'N2', 'N3', 'N3', 'N3', 'MAG', 'DAG', 'TAG', 'TAG', 'acids', 'acids', 'MGTS']
y-text       :   ['°', 'a', 'Relative', '=', 'o', 'Abundance,', 'a', '=', '3', '%', 'a', 'ny', 'e', '36']
y-labels     :   ['30', '25', '20', '15', '10', '5']
legends      :   ['N.', 'gaditana', 'Oil :', '(+)', 'ESI', 'FT-ICR']
Y- Values    :    [5.9, 13.6, 7.7, 3.0, 20.1, 10.6, 9.5, 3.3, 4.2, 4.9, 3.0, 1.7, 0.8] ['300C'] [9.8, 13.6, 8.3, 2.0, 27.6, 9.6, 6.9, 1.7, 4.2, 3.7, 1.7, 0.3, 0.8] [''] [3.2, 8.3, 7.7, 3.6, 11.6, 10.9, 13.0, 5.9, 2.5, 5.2, 4.8, 4.4, 0.8]

[1242] path: Sugano2008_Article_HydrothermalLiqu

number of contours:  11
0 280C
1 [1.3, 28.6, 36.2, 32.0, 35.3, 34.8, 39.4, 34.8, 35.8, 33.2]
2 ['240 C']
3 [1.3, 20.9, 17.9, 19.3, 22.9, 21.7, 21.9, 30.6, 22.0]
file name    :   Yu2014_Article_NutrientFlowsAndQualityOfBio-c-Figure1-1.png
doi          :   
x-labels     :   ['No-Cat', 'Pd/A1203', 'Pd/C', 'Pt/A1203', 'PU/C', 'Raney', 'Ni', 'NaOH', 'Na2C03']
y-text       :   ['So', 'a', '38', 'Bio-crude', 'a', 'ny', '@', '3}', 'oil', 'a', '@', 'yield', 'S', 'S', '(%)', 'a', '»', 'a', '3', 'a', 'a']
y-labels     :   ['55', '50', '45', '40', '35', '30', '25', '20', '15', '10']
legends      :   ['280C']
Y- Values    :   280C [1.3, 28.6, 36.2, 32.0, 35.3, 34.8, 39.4, 34.8, 35.8, 33.2] ['240 C'] [1.3, 20.9, 17.9, 19.3, 22.9, 21.7, 21.9, 30.6, 22.0]

